# Parse Annotations

Join annotations and create dict of dataframes + car counts

In [1]:
from parsingfunc import *

annotations_dir = "../data/VEDAI1024/Annotations1024"
annotations_dict = parse_annotations_to_dict(annotations_dir)


In [2]:
annotations_dict['00000700']

{'annotation_df':      center_x    center_y  orientation  class_label  occluded  \
 0   58.681547  632.567510    -2.844031            1         1   
 1  953.272985  918.473923    -2.807891            1         1   
 
    fully_contained  corner1_x  corner1_y  corner2_x  corner2_y  corner3_x  \
 0                0         41         81         74         39        619   
 1                0        938        976        968        931        906   
 
    corner3_y  corner4_x  corner4_y  
 0        632        645        635  
 1        919        931        918  ,
 'df_length': 2}

# Parse Images

Extract images from Tarfiles

Keep commented - tars have been deleted but images are stored locally

In [4]:
# source_dir = '../raw_img/'
# dest_dir = '../data/VEDAI1024/images/'
# base_filename = 'Vehicules1024'  # Base name of the tar files
# combined_filename = 'Vehicules1024_combined.tar'  # Name for the combined tar file

# # Run the script
# handle_split_tars(source_dir, dest_dir, base_filename, combined_filename)

# Training a Model

In [5]:
import torch
from torchvision import transforms
from PIL import Image

In [6]:
class VEDAI_Dataset(torch.utils.data.Dataset):
    """Parse Data as a PyTorch Dataset"""
    def __init__(self, annotations_dict, img_dir, transform=None):
        self.annotations_dict = annotations_dict
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations_dict)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, f'{str(idx).zfill(8)}_co.png')
        image = Image.open(img_name).convert("RGB")
        
        labels = self.annotations_dict[str(idx).zfill(8)]['annotation_df']
        
        if self.transform:
            image = self.transform(image)
            
        boxes = labels[['corner1_x', 'corner1_y', 'corner2_x', 'corner2_y',
                        'corner3_x', 'corner3_y', 'corner4_x', 'corner4_y']].values
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        
        labels = torch.ones((labels.shape[0],), dtype=torch.int64)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        
        return image, target

transform = transforms.Compose([
    transforms.Resize((800, 800)), 
    transforms.ToTensor()
])



In [7]:
dataset = VEDAI_Dataset(annotations_dict, '../data/VEDAI1024/images/', transform)